# Imports & Data Reading

In [1]:
# Data loading
from google.colab import drive
import zipfile
import pandas as pd
import gdown
import pickle
# Preprocessing
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
# Feature extraction
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaModel, CoherenceModel


# Util
from tqdm import tqdm
from pprint import pprint


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_URL = "https://drive.google.com/file/d/10YvMpIzRSi_GXg91TkrQBL_3JKC5dni7/view?usp=sharing"
PREPROCESSED_ARTICLES_URL = "https://drive.google.com/file/d/1-DQeGkgh-KmOe7uKFEmrj_GCqbKxNJbh/view?usp=sharing"
CORPUS_BOW_URL = "https://drive.google.com/file/d/1-I72t_8jqxIukAdEaxRSBMAYJyfNAT8m/view?usp=sharing"

In [4]:
gdown.download(DATA_URL, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=10YvMpIzRSi_GXg91TkrQBL_3JKC5dni7
To: /content/articles1.csv.zip
100%|██████████| 77.4M/77.4M [00:00<00:00, 86.6MB/s]


'articles1.csv.zip'

In [5]:
DATA_PATH = "articles1.csv.zip"

In [6]:
# Extract the CSV file from the zip archive
with zipfile.ZipFile(DATA_PATH, "r") as zip_ref:
    zip_ref.extractall()

# Load the CSV data into a Pandas DataFrame
df = pd.read_csv("articles1.csv")

In [7]:
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


# Preprocessing

In [8]:
# Extract title and content as its preprocessing will differ from other features
articles =  df['title'] + ' ' + df['content']

In [9]:
articles[0]

'House Republicans Fret About Winning Their Health Care Suit - The New York Times WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporari

In [10]:
def preprocess_articles(articles):

  lemmatizer = WordNetLemmatizer()

  # Remove punctuations and convert text to lowercase
  words = gensim.utils.simple_preprocess(articles, deacc=True)
    
  # Remove stop words & punctuation 
  words = [word for word in words if word not in stop_words and word not in string.punctuation]
  # Lemmatize
  words = [lemmatizer.lemmatize(word) for word in words]

  # remove all non-word and non-space
  # words = re.sub(r'[^\w\s]', '', words) 

  return words


In [11]:
# tqdm.pandas()
# preprocessed_articles = articles.map(preprocess_articles)

Use below cell to save new preprocessed articles instead of loading already saved ones (don't forget to update `PREPROCESSED_ARTICLES_URL` with the new gdrive link)

In [12]:
# from joblib import Parallel, delayed
# preprocessed_articles = Parallel(n_jobs=4)(delayed(preprocess_articles)(article) for article in tqdm(articles, position=0, leave=True))
# preprocessed_articles = pd.Series(preprocessed_articles)
# pickle.dump(preprocessed_articles, open("preprocessed_articles.pkl", "wb"))
# !cp preprocessed_articles.pkl /content/drive/MyDrive/Edu/NLP

In [13]:
gdown.download(PREPROCESSED_ARTICLES_URL, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1-DQeGkgh-KmOe7uKFEmrj_GCqbKxNJbh
To: /content/preprocessed_articles.pkl
100%|██████████| 165M/165M [00:01<00:00, 87.1MB/s]


'preprocessed_articles.pkl'

In [14]:
preprocessed_articles = pickle.load(open("preprocessed_articles.pkl", "rb"))

In [15]:
preprocessed_articles[0]

['house',
 'republican',
 'fret',
 'winning',
 'health',
 'care',
 'suit',
 'new',
 'york',
 'time',
 'washington',
 'congressional',
 'republican',
 'new',
 'fear',
 'come',
 'health',
 'care',
 'lawsuit',
 'obama',
 'administration',
 'might',
 'win',
 'incoming',
 'trump',
 'administration',
 'could',
 'choose',
 'longer',
 'defend',
 'executive',
 'branch',
 'suit',
 'challenge',
 'administration',
 'authority',
 'spend',
 'billion',
 'dollar',
 'health',
 'insurance',
 'subsidy',
 'american',
 'handing',
 'house',
 'republican',
 'big',
 'victory',
 'issue',
 'sudden',
 'loss',
 'disputed',
 'subsidy',
 'could',
 'conceivably',
 'cause',
 'health',
 'care',
 'program',
 'implode',
 'leaving',
 'million',
 'people',
 'without',
 'access',
 'health',
 'insurance',
 'republican',
 'prepared',
 'replacement',
 'could',
 'lead',
 'chaos',
 'insurance',
 'market',
 'spur',
 'political',
 'backlash',
 'republican',
 'gain',
 'full',
 'control',
 'government',
 'stave',
 'outcome',
 'repu

In [16]:
def extract_features(articles, method='tf-idf'):
    if method == 'tf-idf':
        vectorizer = TfidfVectorizer()
        feature_vectors = vectorizer.fit_transform(articles)
        feature_names = vectorizer.get_feature_names_out()
    elif method == 'countvectorizer':
        vectorizer = CountVectorizer()
        feature_vectors = vectorizer.fit_transform(articles)
        feature_names = vectorizer.get_feature_names_out()
    else:
        raise ValueError('Invalid method specified.')
    # TODO:
      # Add word2vec
    
    return feature_vectors, feature_names

In [17]:
# number of unique tokens
total_tokens = preprocessed_articles.explode().nunique()
print(total_tokens)

152705


In [18]:
# create Dictionary from preprocessed articles
dictionary = Dictionary(preprocessed_articles)

# filter out tokens that appear too much or too little
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=150000)

Use below cell to save new corpus bows instead of loading already saved ones (don't forget to update `CORPUS_BOW_URL` with the new gdrive link)

In [19]:
# bag of words corpus from dictionary
#corpus_bow = Parallel(n_jobs=4)(delayed(dictionary.doc2bow)(article) for article in tqdm(preprocessed_articles, position=0, leave=True))
# pickle.dump(corpus_bow, open("corpus_bow.pkl", 'wb'))
# !cp corpus_bow.pkl /content/drive/MyDrive/Edu/NLP

In [20]:
gdown.download(CORPUS_BOW_URL, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1-I72t_8jqxIukAdEaxRSBMAYJyfNAT8m
To: /content/corpus_bow.pkl
100%|██████████| 74.1M/74.1M [00:00<00:00, 160MB/s]


'corpus_bow.pkl'

In [21]:
corpus_bow = pickle.load(open("corpus_bow.pkl", "rb"))

# Modelling

In [22]:
# create TF-IDF Model from corpus
tfidf = TfidfModel(corpus_bow)

# weighted tfidf corpus
corpus_tfidf = tfidf[corpus_bow]

In [23]:
# Apply LDA with 12 topics
lda_model = LdaModel(corpus=corpus_bow,
                     id2word=dictionary,
                     num_topics=12,
                     passes=10,
                     iterations=100)

In [24]:
topics = lda_model.show_topics(num_topics=12, num_words=12)
for topic in topics:
    print(topic)

(0, '0.009*"cnn" + 0.007*"city" + 0.006*"two" + 0.005*"day" + 0.005*"team" + 0.005*"first" + 0.004*"told" + 0.004*"home" + 0.004*"water" + 0.004*"area" + 0.004*"game" + 0.004*"car"')
(1, '0.012*"woman" + 0.011*"twitter" + 0.011*"black" + 0.010*"student" + 0.010*"news" + 0.010*"medium" + 0.007*"breitbart" + 0.007*"white" + 0.007*"university" + 0.006*"school" + 0.006*"right" + 0.006*"group"')
(2, '0.019*"trump" + 0.012*"clinton" + 0.012*"president" + 0.010*"house" + 0.008*"russia" + 0.008*"email" + 0.007*"white" + 0.007*"official" + 0.007*"russian" + 0.007*"investigation" + 0.007*"fbi" + 0.006*"cnn"')
(3, '0.012*"like" + 0.010*"say" + 0.008*"get" + 0.007*"know" + 0.007*"thing" + 0.007*"think" + 0.006*"way" + 0.006*"going" + 0.006*"even" + 0.005*"make" + 0.005*"want" + 0.005*"life"')
(4, '0.012*"u" + 0.010*"attack" + 0.010*"isi" + 0.009*"state" + 0.008*"country" + 0.008*"military" + 0.006*"group" + 0.006*"war" + 0.006*"syria" + 0.006*"force" + 0.006*"united" + 0.005*"north"')
(5, '0.018*"

In [25]:
# Apply LDA with 20 topics
lda_model_20 = LdaModel(corpus=corpus_bow,
                     id2word=dictionary,
                     num_topics=20,
                     passes=15,
                     iterations=100)

In [26]:
topics = lda_model_20.show_topics(num_topics=20, num_words=15)
for topic in topics:
    print(topic)

(0, '0.010*"show" + 0.006*"star" + 0.006*"like" + 0.006*"film" + 0.005*"movie" + 0.004*"first" + 0.004*"book" + 0.004*"world" + 0.003*"best" + 0.003*"series" + 0.003*"story" + 0.003*"music" + 0.003*"character" + 0.003*"made" + 0.003*"work"')
(1, '0.066*"gun" + 0.018*"rio" + 0.016*"olympic" + 0.014*"gold" + 0.014*"second" + 0.011*"olympics" + 0.011*"medal" + 0.010*"control" + 0.010*"la" + 0.010*"amendment" + 0.010*"shooting" + 0.010*"brazil" + 0.009*"award" + 0.009*"breitbart" + 0.009*"american"')
(2, '0.020*"state" + 0.019*"law" + 0.017*"court" + 0.011*"federal" + 0.011*"immigration" + 0.009*"order" + 0.009*"government" + 0.009*"country" + 0.008*"president" + 0.008*"border" + 0.007*"united" + 0.007*"administration" + 0.007*"immigrant" + 0.007*"policy" + 0.007*"obama"')
(3, '0.107*"trump" + 0.028*"clinton" + 0.019*"donald" + 0.016*"president" + 0.015*"campaign" + 0.011*"hillary" + 0.009*"obama" + 0.008*"presidential" + 0.006*"republican" + 0.006*"cnn" + 0.005*"say" + 0.005*"election" + 

In [27]:
# Apply LDA with TF-IDF and 12 topics
lda_model_tfidf = LdaModel(corpus_tfidf,
                           id2word=dictionary,
                           num_topics=12,
                           passes=10,
                           iterations=100)

In [28]:
# Print the top 12 topics
topics = lda_model_tfidf.show_topics(num_topics=12, num_words=15)
for topic in topics:
    print(topic)

(0, '0.016*"cosby" + 0.008*"venezuela" + 0.007*"maduro" + 0.006*"venezuelan" + 0.006*"est" + 0.006*"pipeline" + 0.004*"crutcher" + 0.004*"constand" + 0.004*"dakota" + 0.004*"uranium" + 0.003*"caracas" + 0.003*"accuser" + 0.003*"durst" + 0.003*"ramos" + 0.002*"berman"')
(1, '0.005*"malaysian" + 0.005*"malaysia" + 0.004*"stumpf" + 0.003*"lauer" + 0.002*"nintendo" + 0.002*"watters" + 0.002*"hubble" + 0.002*"indonesian" + 0.002*"kuala" + 0.002*"lumpur" + 0.001*"driverless" + 0.001*"zucker" + 0.001*"chobani" + 0.001*"najib" + 0.001*"dodger"')
(2, '0.014*"nfl" + 0.013*"cia" + 0.011*"dnc" + 0.009*"pope" + 0.007*"nsa" + 0.007*"lgbt" + 0.007*"dc" + 0.006*"shark" + 0.006*"francis" + 0.005*"nypd" + 0.004*"anthem" + 0.004*"jong" + 0.004*"santorum" + 0.004*"boehner" + 0.003*"scarborough"')
(3, '0.014*"rio" + 0.013*"zika" + 0.012*"olympic" + 0.011*"olympics" + 0.010*"virus" + 0.008*"disease" + 0.008*"gold" + 0.008*"medal" + 0.007*"athlete" + 0.007*"cdc" + 0.006*"fda" + 0.006*"infection" + 0.005*"sai

In [35]:
# apply Non-negative matrix Factorizaion with TF-IDF and 15 topics
nmf_model = gensim.models.Nmf(corpus=corpus_tfidf,
                              id2word=dictionary,
                              num_topics = 15,
                              random_state = 42
                              )

In [32]:
nmf_model.show_topics(num_topics=15, num_words=15)

[(0,
  '0.013*"dr" + 0.009*"drug" + 0.008*"patient" + 0.008*"health" + 0.006*"study" + 0.006*"cancer" + 0.006*"case" + 0.006*"north" + 0.005*"medical" + 0.004*"doctor" + 0.004*"university" + 0.004*"could" + 0.004*"disease" + 0.004*"korea" + 0.004*"treatment"'),
 (1,
  '0.031*"school" + 0.014*"student" + 0.013*"child" + 0.013*"woman" + 0.011*"city" + 0.010*"black" + 0.007*"family" + 0.006*"parent" + 0.005*"education" + 0.005*"white" + 0.005*"york" + 0.005*"public" + 0.005*"life" + 0.004*"high" + 0.004*"university"'),
 (2,
  '0.019*"obama" + 0.018*"president" + 0.018*"court" + 0.012*"law" + 0.010*"case" + 0.010*"justice" + 0.008*"federal" + 0.007*"judge" + 0.006*"right" + 0.006*"house" + 0.006*"white" + 0.005*"department" + 0.005*"administration" + 0.005*"supreme" + 0.005*"attorney"'),
 (3,
  '0.027*"police" + 0.016*"officer" + 0.009*"attack" + 0.008*"update" + 0.006*"twitter" + 0.006*"july" + 0.006*"report" + 0.006*"news" + 0.005*"shooting" + 0.005*"man" + 0.005*"shot" + 0.005*"two" + 0

## Model Evaluation

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=nmf_model, texts=corpus_tfidf, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)